# Anna KaRNNa

In this notebook, I'll build a character-wise RNN trained on Anna Karenina, one of my all-time favorite books. It'll be able to generate new text based on the text from the book.

This network is based off of Andrej Karpathy's [post on RNNs](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) and [implementation in Torch](https://github.com/karpathy/char-rnn). Also, some information [here at r2rt](http://r2rt.com/recurrent-neural-networks-in-tensorflow-ii.html) and from [Sherjil Ozair](https://github.com/sherjilozair/char-rnn-tensorflow) on GitHub. Below is the general architecture of the character-wise RNN.

<img src="assets/charseq.jpeg" width="500">

In [1]:
import time
from collections import namedtuple

import numpy as np
import tensorflow as tf

First we'll load the text file and convert it into integers for our network to use.

In [2]:
with open('anna.txt', 'r') as f:
    text=f.read()
vocab = set(text)
vocab_to_int = {c: i for i, c in enumerate(vocab)}
int_to_vocab = dict(enumerate(vocab))
chars = np.array([vocab_to_int[c] for c in text], dtype=np.int32)

In [3]:
text[:100]

'Chapter 1\n\n\nHappy families are all alike; every unhappy family is unhappy in its own\nway.\n\nEverythin'

In [4]:
chars[:100]

array([15, 19, 65, 57, 71, 37, 62, 10, 29, 17, 17, 17, 44, 65, 57, 57,  2,
       10, 13, 65, 26, 27, 14, 27, 37, 52, 10, 65, 62, 37, 10, 65, 14, 14,
       10, 65, 14, 27, 40, 37, 79, 10, 37, 46, 37, 62,  2, 10, 18, 16, 19,
       65, 57, 57,  2, 10, 13, 65, 26, 27, 14,  2, 10, 27, 52, 10, 18, 16,
       19, 65, 57, 57,  2, 10, 27, 16, 10, 27, 71, 52, 10, 82, 12, 16, 17,
       12, 65,  2, 32, 17, 17, 64, 46, 37, 62,  2, 71, 19, 27, 16], dtype=int32)

Now I need to split up the data into batches, and into training and validation sets. I should be making a test set here, but I'm not going to worry about that. My test will be if the network can generate new text.

Here I'll make both input and target arrays. The targets are the same as the inputs, except shifted one character over. I'll also drop the last bit of data so that I'll only have completely full batches.

The idea here is to make a 2D matrix where the number of rows is equal to the number of batches. Each row will be one long concatenated string from the character data. We'll split this data into a training set and validation set using the `split_frac` keyword. This will keep 90% of the batches in the training set, the other 10% in the validation set.

In [5]:
def split_data(chars, batch_size, num_steps, split_frac=0.9):
    """ 
    Split character data into training and validation sets, inputs and targets for each set.
    
    Arguments
    ---------
    chars: character array
    batch_size: Size of examples in each of batch
    num_steps: Number of sequence steps to keep in the input and pass to the network
    split_frac: Fraction of batches to keep in the training set
    
    
    Returns train_x, train_y, val_x, val_y
    """
    
    slice_size = batch_size * num_steps
    n_batches = int(len(chars) / slice_size)
    
    # Drop the last few characters to make only full batches
    x = chars[: n_batches*slice_size]
    y = chars[1: n_batches*slice_size + 1]
    
    # Split the data into batch_size slices, then stack them into a 2D matrix 
    x = np.stack(np.split(x, batch_size))
    y = np.stack(np.split(y, batch_size))
    
    # Now x and y are arrays with dimensions batch_size x n_batches*num_steps
    
    # Split into training and validation sets, keep the virst split_frac batches for training
    split_idx = int(n_batches*split_frac)
    train_x, train_y= x[:, :split_idx*num_steps], y[:, :split_idx*num_steps]
    val_x, val_y = x[:, split_idx*num_steps:], y[:, split_idx*num_steps:]
    
    return train_x, train_y, val_x, val_y

In [6]:
train_x, train_y, val_x, val_y = split_data(chars, 10, 200)

In [7]:
train_x.shape

(10, 178400)

In [8]:
train_x[:,:10]

array([[15, 19, 65, 57, 71, 37, 62, 10, 29, 17],
       [49, 16, 58, 10, 19, 37, 10, 26, 82, 46],
       [10, 72, 65, 71, 72, 19, 27, 16, 66, 10],
       [82, 71, 19, 37, 62, 10, 12, 82, 18, 14],
       [10, 71, 19, 37, 10, 14, 65, 16, 58, 20],
       [10, 61, 19, 62, 82, 18, 66, 19, 10, 14],
       [71, 10, 71, 82, 17, 58, 82, 32, 17, 17],
       [82, 10, 19, 37, 62, 52, 37, 14, 13,  4],
       [19, 65, 71, 10, 27, 52, 10, 71, 19, 37],
       [37, 62, 52, 37, 14, 13, 10, 65, 16, 58]], dtype=int32)

I'll write another function to grab batches out of the arrays made by split data. Here each batch will be a sliding window on these arrays with size `batch_size X num_steps`. For example, if we want our network to train on a sequence of 100 characters, `num_steps = 100`. For the next batch, we'll shift this window the next sequence of `num_steps` characters. In this way we can feed batches to the network and the cell states will continue through on each batch.

In [9]:
def get_batch(arrs, num_steps):
    batch_size, slice_size = arrs[0].shape
    
    n_batches = int(slice_size/num_steps)
    for b in range(n_batches):
        yield [x[:, b*num_steps: (b+1)*num_steps] for x in arrs]

In [10]:
def build_rnn(num_classes, batch_size=50, num_steps=50, lstm_size=128, num_layers=2,
              learning_rate=0.001, grad_clip=5, sampling=False):
        
    if sampling == True:
        batch_size, num_steps = 1, 1

    tf.reset_default_graph()
    
    # Declare placeholders we'll feed into the graph
    with tf.name_scope('inputs'):
        inputs = tf.placeholder(tf.int32, [batch_size, num_steps], name='inputs')
        x_one_hot = tf.one_hot(inputs, num_classes, name='x_one_hot')
    
    with tf.name_scope('targets'):
        targets = tf.placeholder(tf.int32, [batch_size, num_steps], name='targets')
        y_one_hot = tf.one_hot(targets, num_classes, name='y_one_hot')
        y_reshaped = tf.reshape(y_one_hot, [-1, num_classes])
    
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    
    # Build the RNN layers
    with tf.name_scope("RNN_cells"):
        lstm = tf.contrib.rnn.BasicLSTMCell(lstm_size)
        drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
#         cell = tf.contrib.rnn.MultiRNNCell([drop] * num_layers)
        cell = tf.contrib.rnn.MultiRNNCell([tf.contrib.rnn.DropoutWrapper(tf.contrib.rnn.BasicLSTMCell(lstm_size),
            output_keep_prob=keep_prob) for _ in range(num_layers)])
    
    with tf.name_scope("RNN_init_state"):
        initial_state = cell.zero_state(batch_size, tf.float32)

    # Run the data through the RNN layers
    with tf.name_scope("RNN_forward"):
        outputs, state = tf.nn.dynamic_rnn(cell, x_one_hot, initial_state=initial_state)
    
    final_state = state
    
    # Reshape output so it's a bunch of rows, one row for each cell output
    with tf.name_scope('sequence_reshape'):
        seq_output = tf.concat(outputs, axis=1,name='seq_output')
        output = tf.reshape(seq_output, [-1, lstm_size], name='graph_output')
    
    # Now connect the RNN outputs to a softmax layer and calculate the cost
    with tf.name_scope('logits'):
        softmax_w = tf.Variable(tf.truncated_normal((lstm_size, num_classes), stddev=0.1),
                               name='softmax_w')
        softmax_b = tf.Variable(tf.zeros(num_classes), name='softmax_b')
        logits = tf.matmul(output, softmax_w) + softmax_b
        tf.summary.histogram('softmax_w', softmax_w)
        tf.summary.histogram('softmax_b', softmax_b)

    with tf.name_scope('predictions'):
        preds = tf.nn.softmax(logits, name='predictions')
        tf.summary.histogram('predictions', preds)
    
    with tf.name_scope('cost'):
        loss = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y_reshaped, name='loss')
        cost = tf.reduce_mean(loss, name='cost')
        tf.summary.scalar('cost', cost)

    # Optimizer for training, using gradient clipping to control exploding gradients
    with tf.name_scope('train'):
        tvars = tf.trainable_variables()
        grads, _ = tf.clip_by_global_norm(tf.gradients(cost, tvars), grad_clip)
        train_op = tf.train.AdamOptimizer(learning_rate)
        optimizer = train_op.apply_gradients(zip(grads, tvars))
    
    merged = tf.summary.merge_all()
    
    # Export the nodes 
    export_nodes = ['inputs', 'targets', 'initial_state', 'final_state',
                    'keep_prob', 'cost', 'preds', 'optimizer', 'merged']
    Graph = namedtuple('Graph', export_nodes)
    local_dict = locals()
    graph = Graph(*[local_dict[each] for each in export_nodes])
    
    return graph

## Hyperparameters

Here I'm defining the hyperparameters for the network. The two you probably haven't seen before are `lstm_size` and `num_layers`. These set the number of hidden units in the LSTM layers and the number of LSTM layers, respectively. Of course, making these bigger will improve the network's performance but you'll have to watch out for overfitting. If your validation loss is much larger than the training loss, you're probably overfitting. Decrease the size of the network or decrease the dropout keep probability.

In [15]:
batch_size = 50#100
num_steps = 50#100
lstm_size = 256#512
num_layers = 2
learning_rate = 0.001

## Training

Time for training which is is pretty straightforward. Here I pass in some data, and get an LSTM state back. Then I pass that state back in to the network so the next batch can continue the state from the previous batch. And every so often (set by `save_every_n`) I calculate the validation loss and save a checkpoint.

In [16]:
!mkdir -p checkpoints/anna

In [17]:
def train(model, epochs, file_writer):
    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())

        # Use the line below to load a checkpoint and resume training
        #saver.restore(sess, 'checkpoints/anna20.ckpt')

        n_batches = int(train_x.shape[1]/num_steps)
        iterations = n_batches * epochs
        for e in range(epochs):

            # Train network
            new_state = sess.run(model.initial_state)
            loss = 0
            for b, (x, y) in enumerate(get_batch([train_x, train_y], num_steps), 1):
                iteration = e*n_batches + b
                start = time.time()
                feed = {model.inputs: x,
                        model.targets: y,
                        model.keep_prob: 0.5,
                        model.initial_state: new_state}
                summary, batch_loss, new_state, _ = sess.run([model.merged, model.cost, 
                                                              model.final_state, model.optimizer], 
                                                              feed_dict=feed)
                loss += batch_loss
                end = time.time()
                print('Epoch {}/{} '.format(e+1, epochs),
                      'Iteration {}/{}'.format(iteration, iterations),
                      'Training loss: {:.4f}'.format(loss/b),
                      '{:.4f} sec/batch'.format((end-start)))

                file_writer.add_summary(summary, iteration)

In [ ]:
epochs = 5#20
batch_size = 100
num_steps = 100
train_x, train_y, val_x, val_y = split_data(chars, batch_size, num_steps)

for lstm_size in [128,256,512]:
    for num_layers in [1, 2]:
        for learning_rate in [0.002, 0.001]:
            log_string = 'logs/4/lr={},rl={},ru={}'.format(learning_rate, num_layers, lstm_size)
            writer = tf.summary.FileWriter(log_string)
            model = build_rnn(len(vocab), 
                    batch_size=batch_size,
                    num_steps=num_steps,
                    learning_rate=learning_rate,
                    lstm_size=lstm_size,
                    num_layers=num_layers)
            
            train(model, epochs, writer)

Epoch 1/5  Iteration 1/890 Training loss: 4.4204 1.0047 sec/batch
Epoch 1/5  Iteration 2/890 Training loss: 4.4085 0.8002 sec/batch
Epoch 1/5  Iteration 3/890 Training loss: 4.3950 0.7837 sec/batch
Epoch 1/5  Iteration 4/890 Training loss: 4.3759 0.8324 sec/batch
Epoch 1/5  Iteration 5/890 Training loss: 4.3371 0.8020 sec/batch
Epoch 1/5  Iteration 6/890 Training loss: 4.2600 0.8325 sec/batch
Epoch 1/5  Iteration 7/890 Training loss: 4.1707 0.7997 sec/batch
Epoch 1/5  Iteration 8/890 Training loss: 4.0897 0.7947 sec/batch
Epoch 1/5  Iteration 9/890 Training loss: 4.0183 0.8000 sec/batch
Epoch 1/5  Iteration 10/890 Training loss: 3.9582 0.8001 sec/batch
Epoch 1/5  Iteration 11/890 Training loss: 3.9033 0.8217 sec/batch
Epoch 1/5  Iteration 12/890 Training loss: 3.8572 0.8033 sec/batch
Epoch 1/5  Iteration 13/890 Training loss: 3.8157 0.7815 sec/batch
Epoch 1/5  Iteration 14/890 Training loss: 3.7804 0.8313 sec/batch
Epoch 1/5  Iteration 15/890 Training loss: 3.7487 0.8236 sec/batch
Epoc

Epoch 1/5  Iteration 124/890 Training loss: 3.1216 0.5313 sec/batch
Epoch 1/5  Iteration 125/890 Training loss: 3.1187 0.5535 sec/batch
Epoch 1/5  Iteration 126/890 Training loss: 3.1156 0.5552 sec/batch
Epoch 1/5  Iteration 127/890 Training loss: 3.1128 0.5510 sec/batch
Epoch 1/5  Iteration 128/890 Training loss: 3.1100 0.5345 sec/batch
Epoch 1/5  Iteration 129/890 Training loss: 3.1071 0.5431 sec/batch
Epoch 1/5  Iteration 130/890 Training loss: 3.1042 0.5486 sec/batch
Epoch 1/5  Iteration 131/890 Training loss: 3.1014 0.5587 sec/batch
Epoch 1/5  Iteration 132/890 Training loss: 3.0985 0.5459 sec/batch
Epoch 1/5  Iteration 133/890 Training loss: 3.0957 0.5496 sec/batch
Epoch 1/5  Iteration 134/890 Training loss: 3.0928 0.5445 sec/batch
Epoch 1/5  Iteration 135/890 Training loss: 3.0897 0.5588 sec/batch
Epoch 1/5  Iteration 136/890 Training loss: 3.0868 0.5277 sec/batch
Epoch 1/5  Iteration 137/890 Training loss: 3.0839 0.5404 sec/batch
Epoch 1/5  Iteration 138/890 Training loss: 3.08

Epoch 2/5  Iteration 245/890 Training loss: 2.4793 0.6168 sec/batch
Epoch 2/5  Iteration 246/890 Training loss: 2.4781 0.6006 sec/batch
Epoch 2/5  Iteration 247/890 Training loss: 2.4770 0.5987 sec/batch
Epoch 2/5  Iteration 248/890 Training loss: 2.4762 0.6310 sec/batch
Epoch 2/5  Iteration 249/890 Training loss: 2.4755 0.6137 sec/batch
Epoch 2/5  Iteration 250/890 Training loss: 2.4749 0.6175 sec/batch
Epoch 2/5  Iteration 251/890 Training loss: 2.4741 0.6075 sec/batch
Epoch 2/5  Iteration 252/890 Training loss: 2.4733 0.6201 sec/batch
Epoch 2/5  Iteration 253/890 Training loss: 2.4725 0.6165 sec/batch
Epoch 2/5  Iteration 254/890 Training loss: 2.4722 0.6188 sec/batch
Epoch 2/5  Iteration 255/890 Training loss: 2.4714 0.6247 sec/batch
Epoch 2/5  Iteration 256/890 Training loss: 2.4709 0.6216 sec/batch
Epoch 2/5  Iteration 257/890 Training loss: 2.4699 0.6247 sec/batch
Epoch 2/5  Iteration 258/890 Training loss: 2.4691 0.6323 sec/batch
Epoch 2/5  Iteration 259/890 Training loss: 2.46

Epoch 3/5  Iteration 366/890 Training loss: 2.3177 0.7386 sec/batch
Epoch 3/5  Iteration 367/890 Training loss: 2.3162 0.8582 sec/batch
Epoch 3/5  Iteration 368/890 Training loss: 2.3159 0.7455 sec/batch
Epoch 3/5  Iteration 369/890 Training loss: 2.3163 0.7260 sec/batch
Epoch 3/5  Iteration 370/890 Training loss: 2.3186 0.9335 sec/batch
Epoch 3/5  Iteration 371/890 Training loss: 2.3180 0.9681 sec/batch
Epoch 3/5  Iteration 372/890 Training loss: 2.3178 0.7371 sec/batch
Epoch 3/5  Iteration 373/890 Training loss: 2.3174 0.6946 sec/batch
Epoch 3/5  Iteration 374/890 Training loss: 2.3194 0.7255 sec/batch
Epoch 3/5  Iteration 375/890 Training loss: 2.3194 0.7861 sec/batch
Epoch 3/5  Iteration 376/890 Training loss: 2.3183 0.6923 sec/batch
Epoch 3/5  Iteration 377/890 Training loss: 2.3178 0.6775 sec/batch
Epoch 3/5  Iteration 378/890 Training loss: 2.3191 0.7043 sec/batch
Epoch 3/5  Iteration 379/890 Training loss: 2.3185 0.7012 sec/batch
Epoch 3/5  Iteration 380/890 Training loss: 2.31

Epoch 3/5  Iteration 487/890 Training loss: 2.2742 0.7067 sec/batch
Epoch 3/5  Iteration 488/890 Training loss: 2.2736 0.7040 sec/batch
Epoch 3/5  Iteration 489/890 Training loss: 2.2734 0.6859 sec/batch
Epoch 3/5  Iteration 490/890 Training loss: 2.2732 0.7013 sec/batch
Epoch 3/5  Iteration 491/890 Training loss: 2.2729 0.6851 sec/batch
Epoch 3/5  Iteration 492/890 Training loss: 2.2727 0.6835 sec/batch
Epoch 3/5  Iteration 493/890 Training loss: 2.2724 0.6915 sec/batch
Epoch 3/5  Iteration 494/890 Training loss: 2.2723 0.6928 sec/batch
Epoch 3/5  Iteration 495/890 Training loss: 2.2723 0.6677 sec/batch
Epoch 3/5  Iteration 496/890 Training loss: 2.2719 0.7124 sec/batch
Epoch 3/5  Iteration 497/890 Training loss: 2.2718 0.6644 sec/batch
Epoch 3/5  Iteration 498/890 Training loss: 2.2716 0.6869 sec/batch
Epoch 3/5  Iteration 499/890 Training loss: 2.2714 0.6917 sec/batch
Epoch 3/5  Iteration 500/890 Training loss: 2.2711 0.6743 sec/batch
Epoch 3/5  Iteration 501/890 Training loss: 2.27

...

Epoch 5/5  Iteration 720/890 Training loss: 1.8541 4.7299 sec/batch
Epoch 5/5  Iteration 721/890 Training loss: 1.8568 4.7838 sec/batch
Epoch 5/5  Iteration 722/890 Training loss: 1.8550 4.7685 sec/batch
Epoch 5/5  Iteration 723/890 Training loss: 1.8523 4.7219 sec/batch
Epoch 5/5  Iteration 724/890 Training loss: 1.8498 4.7697 sec/batch
Epoch 5/5  Iteration 725/890 Training loss: 1.8497 4.7419 sec/batch
Epoch 5/5  Iteration 726/890 Training loss: 1.8521 4.8079 sec/batch
Epoch 5/5  Iteration 727/890 Training loss: 1.8515 4.7164 sec/batch
Epoch 5/5  Iteration 728/890 Training loss: 1.8500 4.8274 sec/batch
Epoch 5/5  Iteration 729/890 Training loss: 1.8494 4.7285 sec/batch
Epoch 5/5  Iteration 730/890 Training loss: 1.8517 4.7016 sec/batch
Epoch 5/5  Iteration 731/890 Training loss: 1.8516 4.7349 sec/batch
Epoch 5/5  Iteration 732/890 Training loss: 1.8523 4.6887 sec/batch
Epoch 5/5  Iteration 733/890 Training loss: 1.8515 4.7309 sec/batch
Epoch 5/5  Iteration 734/890 Training loss: 1.85

Epoch 5/5  Iteration 841/890 Training loss: 1.8130 4.7455 sec/batch
Epoch 5/5  Iteration 842/890 Training loss: 1.8126 4.7977 sec/batch
Epoch 5/5  Iteration 843/890 Training loss: 1.8121 4.8860 sec/batch
Epoch 5/5  Iteration 844/890 Training loss: 1.8116 4.7329 sec/batch
Epoch 5/5  Iteration 845/890 Training loss: 1.8113 4.7373 sec/batch
Epoch 5/5  Iteration 846/890 Training loss: 1.8111 4.7417 sec/batch
Epoch 5/5  Iteration 847/890 Training loss: 1.8109 4.6978 sec/batch
Epoch 5/5  Iteration 848/890 Training loss: 1.8107 4.7378 sec/batch
Epoch 5/5  Iteration 849/890 Training loss: 1.8107 4.8198 sec/batch
Epoch 5/5  Iteration 850/890 Training loss: 1.8105 4.8519 sec/batch
Epoch 5/5  Iteration 851/890 Training loss: 1.8105 4.7890 sec/batch
Epoch 5/5  Iteration 852/890 Training loss: 1.8102 4.9818 sec/batch
Epoch 5/5  Iteration 853/890 Training loss: 1.8102 5.1345 sec/batch
Epoch 5/5  Iteration 854/890 Training loss: 1.8099 7.6409 sec/batch
Epoch 5/5  Iteration 855/890 Training loss: 1.80

Epoch 1/5  Iteration 72/890 Training loss: 3.2292 11.8643 sec/batch
Epoch 1/5  Iteration 73/890 Training loss: 3.2257 11.5208 sec/batch
Epoch 1/5  Iteration 74/890 Training loss: 3.2221 11.5343 sec/batch
Epoch 1/5  Iteration 75/890 Training loss: 3.2185 11.6628 sec/batch
Epoch 1/5  Iteration 76/890 Training loss: 3.2150 11.5345 sec/batch
Epoch 1/5  Iteration 77/890 Training loss: 3.2113 11.5999 sec/batch
Epoch 1/5  Iteration 78/890 Training loss: 3.2078 11.6216 sec/batch
Epoch 1/5  Iteration 79/890 Training loss: 3.2040 11.4717 sec/batch
Epoch 1/5  Iteration 80/890 Training loss: 3.2000 11.4812 sec/batch
Epoch 1/5  Iteration 81/890 Training loss: 3.1962 11.5479 sec/batch
Epoch 1/5  Iteration 82/890 Training loss: 3.1923 11.8010 sec/batch
Epoch 1/5  Iteration 83/890 Training loss: 3.1885 11.5663 sec/batch
Epoch 1/5  Iteration 84/890 Training loss: 3.1845 11.7164 sec/batch
Epoch 1/5  Iteration 85/890 Training loss: 3.1801 11.7271 sec/batch
Epoch 1/5  Iteration 86/890 Training loss: 3.176

Epoch 2/5  Iteration 192/890 Training loss: 2.1931 11.6471 sec/batch
Epoch 2/5  Iteration 193/890 Training loss: 2.1910 11.6412 sec/batch
Epoch 2/5  Iteration 194/890 Training loss: 2.1885 11.6668 sec/batch
Epoch 2/5  Iteration 195/890 Training loss: 2.1862 11.9721 sec/batch
Epoch 2/5  Iteration 196/890 Training loss: 2.1869 11.5848 sec/batch
Epoch 2/5  Iteration 197/890 Training loss: 2.1854 11.5721 sec/batch
Epoch 2/5  Iteration 198/890 Training loss: 2.1829 11.6676 sec/batch
Epoch 2/5  Iteration 199/890 Training loss: 2.1808 11.6062 sec/batch
Epoch 2/5  Iteration 200/890 Training loss: 2.1809 11.6629 sec/batch
Epoch 2/5  Iteration 201/890 Training loss: 2.1784 11.7844 sec/batch
Epoch 2/5  Iteration 202/890 Training loss: 2.1755 11.7025 sec/batch
Epoch 2/5  Iteration 203/890 Training loss: 2.1734 11.5491 sec/batch
Epoch 2/5  Iteration 204/890 Training loss: 2.1709 11.6519 sec/batch
Epoch 2/5  Iteration 205/890 Training loss: 2.1682 11.7793 sec/batch
Epoch 2/5  Iteration 206/890 Train

Epoch 2/5  Iteration 311/890 Training loss: 2.0127 11.8114 sec/batch
Epoch 2/5  Iteration 312/890 Training loss: 2.0117 11.6770 sec/batch
Epoch 2/5  Iteration 313/890 Training loss: 2.0106 11.5499 sec/batch
Epoch 2/5  Iteration 314/890 Training loss: 2.0095 11.6179 sec/batch
Epoch 2/5  Iteration 315/890 Training loss: 2.0085 11.6836 sec/batch
Epoch 2/5  Iteration 316/890 Training loss: 2.0075 11.5563 sec/batch
Epoch 2/5  Iteration 317/890 Training loss: 2.0066 11.8022 sec/batch
Epoch 2/5  Iteration 318/890 Training loss: 2.0055 11.9776 sec/batch
Epoch 2/5  Iteration 319/890 Training loss: 2.0047 11.7246 sec/batch
Epoch 2/5  Iteration 320/890 Training loss: 2.0036 11.5758 sec/batch
Epoch 2/5  Iteration 321/890 Training loss: 2.0025 11.7774 sec/batch
Epoch 2/5  Iteration 322/890 Training loss: 2.0015 11.5681 sec/batch
Epoch 2/5  Iteration 323/890 Training loss: 2.0003 11.6130 sec/batch
Epoch 2/5  Iteration 324/890 Training loss: 1.9995 11.7342 sec/batch
Epoch 2/5  Iteration 325/890 Train

In [44]:
tf.train.get_checkpoint_state('checkpoints/anna')

## Sampling

Now that the network is trained, we'll can use it to generate new text. The idea is that we pass in a character, then the network will predict the next character. We can use the new one, to predict the next one. And we keep doing this to generate all new text. I also included some functionality to prime the network with some text by passing in a string and building up a state from that.

The network gives us predictions for each character. To reduce noise and make things a little less random, I'm going to only choose a new character from the top N most likely characters.



In [17]:
def pick_top_n(preds, vocab_size, top_n=5):
    p = np.squeeze(preds)
    p[np.argsort(p)[:-top_n]] = 0
    p = p / np.sum(p)
    c = np.random.choice(vocab_size, 1, p=p)[0]
    return c

In [41]:
def sample(checkpoint, n_samples, lstm_size, vocab_size, prime="The "):
    prime = "Far"
    samples = [c for c in prime]
    model = build_rnn(vocab_size, lstm_size=lstm_size, sampling=True)
    saver = tf.train.Saver()
    with tf.Session() as sess:
        saver.restore(sess, checkpoint)
        new_state = sess.run(model.initial_state)
        for c in prime:
            x = np.zeros((1, 1))
            x[0,0] = vocab_to_int[c]
            feed = {model.inputs: x,
                    model.keep_prob: 1.,
                    model.initial_state: new_state}
            preds, new_state = sess.run([model.preds, model.final_state], 
                                         feed_dict=feed)

        c = pick_top_n(preds, len(vocab))
        samples.append(int_to_vocab[c])

        for i in range(n_samples):
            x[0,0] = c
            feed = {model.inputs: x,
                    model.keep_prob: 1.,
                    model.initial_state: new_state}
            preds, new_state = sess.run([model.preds, model.final_state], 
                                         feed_dict=feed)

            c = pick_top_n(preds, len(vocab))
            samples.append(int_to_vocab[c])
        
    return ''.join(samples)

In [44]:
checkpoint = "checkpoints/anna/i3560_l512_1.122.ckpt"
samp = sample(checkpoint, 2000, lstm_size, len(vocab), prime="Far")
print(samp)

Farlathit that if had so
like it that it were. He could not trouble to his wife, and there was
anything in them of the side of his weaky in the creature at his forteren
to him.

"What is it? I can't bread to those," said Stepan Arkadyevitch. "It's not
my children, and there is an almost this arm, true it mays already,
and tell you what I have say to you, and was not looking at the peasant,
why is, I don't know him out, and she doesn't speak to me immediately, as
you would say the countess and the more frest an angelembre, and time and
things's silent, but I was not in my stand that is in my head. But if he
say, and was so feeling with his soul. A child--in his soul of his
soul of his soul. He should not see that any of that sense of. Here he
had not been so composed and to speak for as in a whole picture, but
all the setting and her excellent and society, who had been delighted
and see to anywing had been being troed to thousand words on them,
we liked him.

That set in her money at th

In [43]:
checkpoint = "checkpoints/anna/i200_l512_2.432.ckpt"
samp = sample(checkpoint, 1000, lstm_size, len(vocab), prime="Far")
print(samp)

Farnt him oste wha sorind thans tout thint asd an sesand an hires on thime sind thit aled, ban thand and out hore as the ter hos ton ho te that, was tis tart al the hand sostint him sore an tit an son thes, win he se ther san ther hher tas tarereng,.

Anl at an ades in ond hesiln, ad hhe torers teans, wast tar arering tho this sos alten sorer has hhas an siton ther him he had sin he ard ate te anling the sosin her ans and
arins asd and ther ale te tot an tand tanginge wath and ho ald, so sot th asend sat hare sother horesinnd, he hesense wing ante her so tith tir sherinn, anded and to the toul anderin he sorit he torsith she se atere an ting ot hand and thit hhe so the te wile har
ens ont in the sersise, and we he seres tar aterer, to ato tat or has he he wan ton here won and sen heren he sosering, to to theer oo adent har herere the wosh oute, was serild ward tous hed astend..

I's sint on alt in har tor tit her asd hade shithans ored he talereng an soredendere tim tot hees. Tise sor 

In [46]:
checkpoint = "checkpoints/anna/i600_l512_1.750.ckpt"
samp = sample(checkpoint, 1000, lstm_size, len(vocab), prime="Far")
print(samp)

Fard as astice her said he celatice of to seress in the raice, and to be the some and sere allats to that said to that the sark and a cast a the wither ald the pacinesse of her had astition, he said to the sount as she west at hissele. Af the cond it he was a fact onthis astisarianing.


"Or a ton to to be that's a more at aspestale as the sont of anstiring as
thours and trey.

The same wo dangring the
raterst, who sore and somethy had ast out an of his book. "We had's beane were that, and a morted a thay he had to tere. Then to
her homent andertersed his his ancouted to the pirsted, the soution for of the pirsice inthirgest and stenciol, with the hard and and
a colrice of to be oneres,
the song to this anderssad.
The could ounterss the said to serom of
soment a carsed of sheres of she
torded
har and want in their of hould, but
her told in that in he tad a the same to her. Serghing an her has and with the seed, and the camt ont his about of the
sail, the her then all houg ant or to hus

In [47]:
checkpoint = "checkpoints/anna/i1000_l512_1.484.ckpt"
samp = sample(checkpoint, 1000, lstm_size, len(vocab), prime="Far")
print(samp)

Farrat, his felt has at it.

"When the pose ther hor exceed
to his sheant was," weat a sime of his sounsed. The coment and the facily that which had began terede a marilicaly whice whether the pose of his hand, at she was alligated herself the same on she had to
taiking to his forthing and streath how to hand
began in a lang at some at it, this he cholded not set all her. "Wo love that is setthing. Him anstering as seen that."

"Yes in the man that say the mare a crances is it?" said Sergazy Ivancatching. "You doon think were somether is ifficult of a mone of
though the most at the countes that the
mean on the come to say the most, to
his feesing of
a man she, whilo he
sained and well, that he would still at to said. He wind at his for the sore in the most
of hoss and almoved to see him. They have betine the sumper into at he his stire, and what he was that at the so steate of the
sound, and shin should have a geest of shall feet on the conderation to she had been at that imporsing the